In [18]:
from sklearn.linear_model import LinearRegression

In [ ]:
### В этой тетрадке мы объеденили нашу идею, для более понятного её представления.

##### Идея состоит в том, чтобы для работодателей рекомендовать пользователей гитхаба с примерной ценой их работы.

### Загружаем модель

Для решения задачи мы сначала сжали пространство признаков при помощи PCA с 16 компонентами, а потом применили SVR в с rbf kernel над преобразованными данными. Количество компонент и классификатор выбирались по кроссвалидации (см. [ActuallytryingToFitSomeModel.ipynb](https://github.com/Quest753/hackathon2016/blob/master/ActualtryingTofit.ipynb))


<img src="http://what-when-how.com/wp-content/uploads/2011/07/tmp691429_thumb.jpg" alt="Drawing" style="width: 640px; height: 480px"/>


In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.decomposition import pca

## Для обучения использовались данные с headhunter'a(см [workWithHH.ipynb]())

In [2]:
with open('data.pickle', 'rb') as f:
    svr = pickle.load(f)
with open('pca.pickle','rb') as f:
    pca = pickle.load(f)


In [3]:
def predict(data):
    '''
        Функция предсказывающая цену для каждого пользователя 
            на основе важности требуемых языков программирования.
        data - входные данные для предсказания цены.
        Возвращает предсказанную цену.
    '''
    #уменьшение размерности входных данных
    data_pca = pca.transform(data)

    #предсказание логарифма зарплаты при помощи RBF SVM
    prediction = svr.predict(data_pca)

    #пересчитываем логарифм зарплаты обратно в зарплату 
    salary = np.exp(prediction)
    
    return salary



### Считаем тестовые данные
Для того, чтобы предсказываемая цена зависила только от тех языков программирования, которые требуются работодателю, занулим все остальные языки. Тестовые данные берутся из гитхаба, процесс сбора и анализа полученной информации показан в файлах:
* сбор :  [git.ipynb](https://github.com/Quest753/hackathon2016/blob/master/git.ipynb) 
* анализ : [getDataFromGit.ipynb](https://github.com/Quest753/hackathon2016/blob/master/getDataFromGit.ipynb)

In [12]:
##читаем данные - предобработанную информацию о людях с гита для предсказания цены их работы.
new_data = pd.read_csv("./github/newGitHubUsers.csv")
#передваемые параметры с сайта(Параметры, которые указывает работодатель)
parameters = ["SQL", 100000]
#запонляем всё нулями because reasons
# for col in new_data.columns:
#     if col not in parameters:
#         new_data[col] = np.zeros(len(new_data))

In [5]:
#Предсказание модели
salary = predict(new_data)

### Составляем таблицу для определения подходящих по цене программистов
Для этого соенияем данные о пользователе с гитхаба( Количество фолловеров, количество просмотров, ссылку на гитхаб) и предсказанную цену.
Затем сортируем выводим пользователей по количеству фолловеров и просмотров. 

In [42]:
#полная информация о пользователях с гита
ranging = pd.read_csv('github/rangingGitLink.csv')
ranging['Salary'] = salary
string = parameters
##выбор пользователей, подходящих под запрос
ans = ranging[(ranging.Salary <= string[-1] * 1.5) & (ranging.Salary >= string[-1] * 0.5)]
ans = ans.sort_values(['Salary', 'Watchers'])[::-1]
#ans.to_csv('answer11111-.csv')

In [43]:
##запрос ["CSS","HTML","JavaScript",80000]
ans

,Followers,Watchers,Link,Salary


In [20]:
# ["C++","C#",90000]
# коммент что это такое
ans.head()

,Followers,Watchers,Link,Salary


In [42]:
# ["SQL", "Go", "Java", 90000]
# коммент что это такое
ans.head()

,Followers,Watchers,Link,Salary
14891,88,128,https://github.com/jenkinsci,91145.125219
24352,542,29,https://github.com/apache,88200.962661
19000,12,51,https://github.com/mulesoft,87322.276803
14084,113,34,https://github.com/ndeloof,84554.531801
15176,9,63,https://github.com/carrot-garden,84379.118651
